<a href="https://colab.research.google.com/github/e-neuman/pandemic_model/blob/master/Pandemic_in_population_with_heterogeneous_risk_and_exposure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandemic in population with heterogeneous risk and exposure

Does accounting for heterogeneous fatality rates in the population change how to best approach a pandemic?

The model aims to:
- be as simple as possible (there are no incubation periods, for instance)
- be fast, to be able to run sensitivity analysis iteratively

*Disclaimer: I am not an expert on the subject*

In [0]:
# Model requirements:

import numpy as np

# Plotting and analysis:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Model

Inputs:
- `R` (dict of matrices) Virus transmission rate across groups:
    
    R[k][i,j] = Number of new people in group j that each infected person in group i ___exposes___ at time >= k.
    
    This model assumes that every exposed person becomes infected unless they already have/had the virus.
    
    In a stochastic model, this would be:
    
    p(person in group i ___infects___ person in group j | person in i is infected AND person in group j is not infected) at time >= k
    
    
- `P` (dict of arrays) Fatality rates of each risk group:

    P[k][i] = Number of people in group i that die per infected person.
    
    In a stochastic model, this would be:
    
    p(person in group i dies | person is infected) at time >= k
    

- `N0` (array) Initial number of non infected people in each group
- `I0` (array) Initial number of infected people in each group
- `time_infectious` (int) number of units of time an infected person can transmit the virus
- `time_limit` (int) max units of time simulated
- `break_rate` (optional - float) rate of new deaths at which to stop simulation (optimization for grid calculations purposes)

Outputs:
- `N` (matrix) N[i,j] = Number of non infected people in group j at time i
- `I` (matrix) I[i,j] = Number of infected people in group j at time i
- `F` (matrix) F[i,j] = Number of total fatalities in group j by time i
- `G` (matrix) G[i,j] = Number of recovered people in group j at time i

In [0]:
def simulate(R, P, N0, I0, time_infectious, time_limit, break_rate=0):
    n = len(N0)

    N = np.zeros((time_limit, n)) # Not infected population
    dI = np.zeros((time_limit, n)) # Newly infected population
    I = np.zeros((time_limit, n)) # Total infected population
    F = np.zeros((time_limit, n)) # Fatalities
    G = np.zeros((time_limit, n)) # Recovered

    N[0, :] = N0
    I[0, :] = I0
    dI[0, :]= I[0, :]
    dF = np.array([0, 0])
    dG = np.array([0, 0])
    dFp = 0

    for i in range(time_limit-1):
        R_current = np.array(R[[k for k in R.keys() if k <= i][-1]])
        
        dI[i+1, :] = np.matmul(I[i, :], R_current) * N[i, :]\
                     / time_infectious / (N[0,:] + I[0,:]) 

        if i >= time_infectious:
            P_current = np.array(P[[k for k in P.keys() if k <= i][-1]]).reshape(1, n)
            dF = dI[i-time_infectious, :] * P_current
            dG = dI[i-time_infectious, :] - dF
        N[i+1, :] = N[i, :] - dI[i+1, :]
        I[i+1, :] = I[i, :] + dI[i+1, :] - dF - dG
        F[i+1, :] = F[i, :] + dF
        G[i+1, :] = G[i, :] + dG
        
        # stop simulating when death rates are decreasing and below threshold
        if (dF.sum() < dFp) and (dFp < break_rate):
            break
        dFp = dF.sum()

    return N[:i+2, :], I[:i+2, :], F[:i+2, :], G[:i+2, :]

### Default parameters

This numbers are not representative of any population / virus. This is an hypothetical population with 2 distinct (iterconnected) groups, one with high fatality rate, the other with low fatality rate

In [0]:
default_params = {
    'P': {0: [0, 1]},           # group 0 has 0.01% chance of dying, group 1 has 100% - from time 0 onwards
    'N0': [79_999.2, 19_999.8], # there is a 80%/20% split of the population between groups
    'I0': [1, 0],               # start with 1 infectious person from group 0
    'time_infectious': 10,      # takes 10 units of time for an infected person to stop spreading this virus
    'time_limit': 180,          # run simulation for 180 units of time
    'break_rate': 0             # no break rate
}
params = lambda x: {**default_params, **x}

In [0]:
def plot_outbrake(**kwargs):

    def to_df(M, label): # helper function
        df_M = pd.DataFrame(M).unstack()\
                              .reset_index()\
                              .rename(columns={'level_0': 'group', 
                                               'level_1': 'time', 
                                               0: 'per100k'})
        
        df_M['label'] = label
        return df_M


    N, I, F, G = simulate(**{**default_params, **kwargs})

    df = pd.concat([to_df(N, 'never infected'),
                    to_df(F, 'fatalities'),
                    to_df(I, 'infected'),
                    to_df(G, 'recovered')], axis=0)
    return px.line(df, 'time', 'per100k', 'group', color='label')

# Scenarios

## Homogeneuos quarentine

In [0]:
plot_outbrake(R={0: [[4, 1], [4, 1]], # initial inaction - equivalent R0 would be 5
                 20: [[0.4, 0.1], [0.4, 0.1]], # strict homogeneus quarentine - equivalent R0 would be .5
                 50: [[0.7, 0.7], [0.2, 0.2]]}) # less strict quarentine - equivalent R0 would be .9

### Sensitivity to policy strictness

In [0]:
strict_R0 = [0.1+0.05*i for i in range(14)]
strict_len = [5+i*2 for i in range(20)]

go.Figure(data=
    go.Contour(z=[[simulate(**{**default_params,
                               'R': {0: [[4, 1], [4, 1]],
                                     14: [[0.8*r, 0.2*r], [0.8*r, 0.2*r]], 
                                     14+l: [[0.7, 0.2], [0.7, 0.2]]}})[2][-1,:].sum()
                   for l in strict_len] for r in strict_R0],
               x=strict_len,
               y=strict_R0,
               ncontours=100,
               line={'color': 'white'})
         ).update_xaxes(title_text='Duration').update_yaxes(title_text='R0').show()

A stricter policy means it can be shorter for the same effects

## Ideal herd immunity

In [0]:
plot_outbrake(R={0: [[4, 1], [1, 4]], # initial inaction
                 14: [[4, 0], [0, 0]], # super "cuckoon"
                 74: [[4, 1], [4, 1]]})  # Back to normal after 60 time units

#### Sensitivity to R[0,1]  and R[0,0]

In [0]:
r01_stops = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2]
r00_stops = [0.5, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 6, 7]

In [0]:
go.Figure(data=
    go.Contour(z=[[simulate(**{**default_params,
                               'R': {0: [[4, 1], [4, 1]],
                                     14: [[r00, r01], [4*r01, 0.2]], 
                                     74: [[4, 1], [4, 1]]}})[2][-1,:].sum()
                   for r01 in r01_stops] for r00 in r00_stops],
               x=r01_stops,
               y=r00_stops,
               ncontours=100,
               colorscale='Hot', # different color scale to emphasize new color scale
               line={'color': 'white'})
         ).update_xaxes(title_text='R[0,1]').update_yaxes(title_text='R[0,0]').show()

In this graph there are two sections:
- upper section (above 3 for r00), death rate depends on r01 but not on r00: herd immunity was successful
- lower section, death rate is very high and independent of r01: herd immunity failed because controls were released too early.

Also, notice the CHANGE in scale!

#### Sensitivity to R[0,0]  and "cuckoon" duration

In [0]:
r00_stops = [0.5, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 6, 7]
strict_len = [20+i*10 for i in range(8)]

go.Figure(data=
    go.Contour(z=[[simulate(**{**default_params,
                               'R': {0: [[4, 1], [4, 1]],
                                     14: [[r00, 0.01], [4*0.01, 0.2]], 
                                     14+l: [[4, 1], [4, 1]]}})[2][-1,:].sum()
                   for l in strict_len] for r00 in r00_stops],
               x=strict_len,
               y=r00_stops,
               ncontours=100,
               colorscale="Hot",
               line={'color': 'white'})
         ).update_xaxes(title_text='Duration').update_yaxes(title_text='R[0,0]').show()

If this model represents the viral propagation properly, for "herd immunity" to work, we need
 - excelent identification of high risk population: each false negative is a very likely death.
 - extreme isolation of this population segment, are such low exposures even possible?
 - increase exposure of low risk population: does this render herd immunity incompatible with quarantine?

**Comments, copies, changes are more than welcome!**